# 循环神经网络
:label:`sec_rnn`


在 :numref:`sec_language_model` 我们引入了 $n$-gram 模型，其中单词 $x_t$ 在时间步长 $t$ 处的条件概率仅取决于前面的 $n-1$ 
如果我们想将早于时间步长 $t-(n-1)$ 的单词对 $x_t$ 的可能影响结合起来，
我们需要增加 $n$.
但是，模型参数的数量也会随之呈指数增长，因为我们需要为词汇集 $\mathcal{V}$ 存储 $|\mathcal{V}|^n$ 个数字。
因此，与其对 $P(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$ 建模，不如使用隐变量模型：

$$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1}),$$

其中，$h_{t-1}$ 是一个 *隐藏状态* (也称为隐藏变量)，它存储时间步长 $t-1$ 之前的序列信息。
一般来说，
可以基于当前输入 $x{t}$ 和之前的隐藏状态 $h{t-1}$ 计算任何时间步骤 $t$ 的隐藏状态：

$$h_t = f(x_{t}, h_{t-1}).$$
:eqlabel:`eq_ht_xt`

对于足够强大的函数 $f$ in :eqref:`eq_ht_xt`，隐变量模型不是近似值。毕竟，$h_t$可以简单地存储迄今为止观察到的所有数据。
然而，它可能会使计算和存储都变得昂贵。

回想一下，我们在：:numref:`chap_perceptrons`，中讨论过带隐藏单位的隐藏层。
值得注意的是,
隐藏层和隐藏状态指的是两个非常不同的概念。
如前所述，隐藏层是在从输入到输出的路径上从视图中隐藏的层。
从技术上讲，隐藏状态是我们在给定步骤中所做任何事情的*输入*，
它们只能通过查看以前时间步长的数据来计算。

*递归神经网络*（RNN）是具有隐藏状态的神经网络。在介绍RNN模型之前，我们首先回顾一下 :numref:`sec_mlp` 中介绍的MLP模型。

## 没有隐藏状态的神经网络

让我们看一看具有单个隐藏层的MLP。
让隐藏层的激活函数为 $\phi$ 。
给定一小批示例 $\mathbf{X} \in \mathbb{R}^{n \times d}$ 批次大小为 $n$ 输入为 $d$ ，
隐藏层的输出 $\mathbf{H} \in \mathbb{R}^{n \times h}$ 计算如下：

$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).$$
:eqlabel:`rnn_h_without_state`

在 :eqref:`rnn_h_without_state` 我们权重参数 $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$，
偏差参数 $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$ 以及隐藏层的隐藏单元数 $h$ ，
因此，在求和期间应用广播 (参加 :numref:`subsec_broadcasting`)
接下来，将隐藏变量 $\mathbf{H}$ 作输出层的输入。输出层由

$$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q,$$

其中 $\mathbf{O} \in \mathbb{R}^{n \times q}$ 是输出变量，$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ 是权重参数，
$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ 是输出层的偏差参数。如果是分类问题，我们可以使用 $\text{softmax}(\mathbf{O})$ 
来计算输出类别的概率分布。

这完全类似于我们之前在：:numref:`sec_sequence` 中解决的回归问题，因此我们省略了细节。
可以这样说，我们可以随机选取特征标签对，并通过自动微分和随机梯度下降来学习网络的参数。

## 隐藏状态的递归神经网络
:label:`subsec_rnn_w_hidden_states`

当我们有隐藏状态时，情况就完全不同了。让我们更详细地看一下结构。

假设我们有
一小批输入
$\mathbf{X}_t \in \mathbb{R}^{n \times d}$
在时间步长 $t$.
换句话说，
对于 $n$ 序列示例的小批量，
$\mathbf{X}_t$ 的每一行对应于序列中时间步 $t$ 的一个示例。
下一个，用
$\mathbf{H}_t  \in \mathbb{R}^{n \times h}$ 表示时间步长 $t$ 的隐藏变量。
与MLP不同的是，这里我们保存了前一时间步中的隐藏变量 $\mathbf{H}_{t-1}$ 并在 $\mathbb{R}^{h \times h}$ 
中引入了一个新的权重参数 $\mathbf{W}_{hh}$ 来描述如何在当前时间步中使用前一时间步的隐藏变量。
具体而言，当前时间步长的隐藏变量的计算由当前时间步长的输入以及前一时间步长的隐藏变量确定：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$
:eqlabel:`rnn_h_with_state`

与 :eqref:`rnn_h_without_state` 相比，:eqref:`rnn_h_with_state` 增加了一个术语 $\mathbf{H}_{t-1} \mathbf{W}_{hh}$，因此
实例化 :eqref:`eq_ht_xt`。

根据相邻时间步长的隐藏变量 $\mathbf{H}_t$ 和 $\mathbf{H}_{t-1}$ 之间的关系，
我们知道，这些变量捕捉并保留了序列的历史信息，直到它们的当前时间步，就像神经网络当前时间步的状态或记忆一样。因此，这种隐藏变量称为
*隐藏状态*。
由于隐藏状态使用当前时间步长中前一时间步长的相同定义，因此：
:eqref:`rnn_h_with_state` 的计算是 *递归的*。因此，具有隐藏状态的神经网络
基于递归计算，命名为
*递归神经网络*。
执行以下操作的层
带状态的 :eqref:`rnn_h_with_state`
在RNNs中
称为 *重复层*。

构建RNN有许多不同的方法。
具有隐藏状态的rnn非常常见，其定义为 :eqref:`rnn_h_with_state`
对于时间步长 $t$，
输出层的输出类似于MLP中的计算：

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

RNN的参数
包括权重 $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$,
和偏差 $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$
对于隐藏层，
与权重 $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$
和偏差 $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$
输出层的一部分。
值得一提的是
即使在不同的时间步，
RNN始终使用这些模型参数。
因此，RNN的参数化成本
不会随着时间步数的增加而增加。

:numref:`fig_rnn` 说明了rnn在三个相邻时间步的计算逻辑。
在任何时候，步骤 $t$,
隐藏状态的计算可被视为：
i) 在当前时间步骤 $t$ 连接输入 $\mathbf{X}_t$ ，在上一时间步骤 $t-1$ 连接隐藏状态 $\mathbf{H}_{t-1}$ ;
ii) 使用激活函数 $\phi$ 连接结果馈送到完全连接的层。
这种完全连接层的输出是当前时间步长 $t$ 的隐藏状态 $\mathbf{H}_t$
在这种情况下，
型参数是 $\mathbf{W}_{xh}$ 和 $\mathbf{W}_{hh}$ 的串联，以及 $\mathbf{b}_h$ 的偏差，所有这些参数都来自 :eqref:`rnn_h_with_state`.
当前时间步骤 $t$ $\mathbf{H}_t$ 的隐藏状态将参与计算下一时间步骤 $t+1$ 的隐藏状态 $\mathbf{H}_{t+1}$ 。

更重要的是，$\mathbf{H}_t$ 也将是
输入完全连接的输出层
计算输出
当前时间步长 $t$ 的 $\mathbf{O}_t$ 。

![具有隐藏状态的RNN](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/rnn.svg)
:label:`fig_rnn`

我们刚才提到 $\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$ 对于隐藏状态的计算等价于
矩阵乘法
$\mathbf{X}_t$ 和 $\mathbf{H}_{t-1}$ 的串联
和
$\mathbf{W}_{xh}$ 和 $\mathbf{W}_{hh}$ 的串联
虽然这可以用数学证明，
在下面，我们只使用一个简单的代码片段来说明这一点。
首先，
我们定义了矩阵 `X`, `W_xh`, `H`, 和 `W_hh`，它们的形状分别是(3, 1), (1, 4), (3, 4), 和 (4, 4)
分别将 `X` 乘以 `W_xh` ，和 `H` 乘以 `W_hh`，然后将这两个乘法相加，
我们得到一个形状为（3，4）的矩阵。


In [ ]:
%load ../utils/djl-imports

In [ ]:
NDManager manager = NDManager.newBaseManager();

In [ ]:
NDArray X = manager.randomNormal(0, 1, new Shape(3, 1), DataType.FLOAT32);
NDArray W_xh = manager.randomNormal(0, 1, new Shape(1, 4), DataType.FLOAT32);
NDArray H = manager.randomNormal(0, 1, new Shape(3, 4), DataType.FLOAT32);
NDArray W_hh = manager.randomNormal(0, 1, new Shape(4, 4), DataType.FLOAT32);
X.dot(W_xh).add(H.dot(W_hh))

现在我们连接矩阵 `X` 和 `H`
沿着立柱（轴1），
和 矩阵
`W_xh` 和 `W_hh` 沿行（轴0）。
这两个串联
导致
形状矩阵 (3, 5)
形状分别为 (5, 4)。
将这两个串联矩阵相乘，
我们得到形状(3, 4)的相同输出矩阵
如上所述。


In [ ]:
X.concat(H, 1).dot(W_xh.concat(W_hh, 0))

## 基于RNN的字符级语言模型

回想一下，对于 :numref:`sec_language_model` 中的语言建模，
我们的目标是基于
当前和过去的tokens，
因此，我们将原始序列移动一个标记
作为标签。
Bengio 等人首先提出
使用神经网络进行语言建模：引用 :cite:`Bengio.Ducharme.Vincent.ea.2003`.
下面我们将说明如何使用RNN构建语言模型。
让minibatch大小为1，文本的顺序为 "machine" 。
为了简化后续章节中的培训，
我们将文本标记为字符而不是单词
并考虑一个*字符级语言模型*。
:numref:`fig_rnn_train` 演示如何通过用于字符级语言建模的rnn根据当前和以前的字符预测下一个字符。

![基于RNN的字符级语言模型。输入和标签序列分别为"machin"和"achine"。](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/rnn-train.svg)
:label:`fig_rnn_train`

在培训过程中，
我们在每个时间步对输出层的输出运行softmax操作，然后使用交叉熵损失计算模型输出和标签之间的误差。
由于在隐藏层中反复计算隐藏状态，时间步长3的输出在：
:numref:`fig_rnn_train`，
$\mathbf{O}_3$，由文本序列"m"，"a"，和"c"确定。由于训练数据中序列的下一个字符是"h"，
因此时间步长3的丢失将取决于基于该时间步长的特征序列
"m"， "a"， "c"和标签 "h" 生成的下一个字符的概率分布。

在实践中，每个token由 $d$-维向量表示，我们使用批次大小 $n>1$。
因此，时间步长 $t$ 处的输入 $\mathbf X_t$ 将是一个 $n\times d$ 的矩阵，这与我们在
:numref:`subsec_rnn_w_hidden_states` 隐藏状态，中讨论的内容相同。


## 困惑
:label:`subsec_perplexity`

最后，让我们讨论如何度量语言模型质量，这将在后面的部分中用于评估基于RNN的模型。
一种方法是检查文本是否令人惊讶。
一个好的语言模型能够用
我们接下来将看到的高精度标记。
考虑不同语言模型所提出的短语"下雨"的连续性：

1. "外面在下雨"
1. "正在下香蕉树雨"
1. "天在下雨；kcj在下雨"

就质量而言，示例1显然是最好的。这些话很有道理，逻辑上也很连贯。
虽然它可能无法准确反映哪个词在语义上跟在后面（"在旧金山"和"在冬天"可能是完全合理的扩展），但该模型能够捕捉到哪个词跟在后面。
示例2由于产生了一个无意义的扩展而变得相当糟糕。尽管如此，至少该模型已经学会了如何拼写单词以及单词之间的某种程度的相关性。最后，示例3指出了一个训练不好的模型，该模型不能正确地拟合数据。

我们可以通过计算序列的可能性来衡量模型的质量。
不幸的是，这是一个难以理解和比较的数字。
毕竟，较短的序列比较长的序列更有可能发生，
因此，对托尔斯泰巨著的模型进行了评估
*战争与和平的可能性必然比圣埃克苏佩里的中篇小说《小王子》要小得多。* 缺少的是一个平均值。

信息理论在这里很有用。
我们定义了熵、超熵和交叉熵
当我们引入softmax回归时
(:numref:`subsec_info_theory_basics`)
更多关于信息论的内容，请参见 [信息论在线附录](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html).
如果我们想压缩文本，我们可以询问
根据当前令牌集预测下一个tokens.
更好的语言模型应该允许我们更准确地预测下一个标记。
因此，它应该允许我们在压缩序列时花费更少的位。
所以我们可以用平均的交叉熵损失来衡量它
在序列的所有 $n$ 标记上：

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$
:eqlabel:`eq_avg_ce_for_lm`

其中， $P$ 由语言模型给出， $x_t$ 是在序列的时间步 $t$ 观察到的实际标记。
这使得不同长度的文档的性能具有可比性。出于历史原因，自然语言处理领域的科学家更喜欢使用一种称为“困惑”的量。
简而言之，它是
:eqref:`eq_avg_ce_for_lm`:

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

困惑可以被最好地理解为当我们决定下一个选择哪个标记时，真实选择数量的调和平均值。让我们看看一些案例：

* 在最佳情况下，该模型始终完美地将标签令牌的概率估计为1。在这种情况下，模型的复杂性是1。
* 在最坏的情况下，模型总是预测标签令牌的概率为0。在这种情况下，困惑是正无穷大。
* 在起点时，该模型预测词汇表中所有可用标记的均匀分布。在这种情况下，困惑等于词汇表中唯一标记的数量。事实上，如果我们在不进行任何压缩的情况下存储序列，这将是我们所能做的最好的编码。因此，这提供了一个重要的上限，任何有用的模型都必须超越这个上限。

在以下部分中，我们将实现RNN
对于字符级语言模型和使用困惑
评估这些模型。


## 总结

* 对隐藏状态使用递归计算的神经网络称为递归神经网络（RNN）。
* RNN的隐藏状态可以捕获当前时间步之前序列的历史信息。
* RNN模型参数的数量不会随着时间步数的增加而增加。
* 我们可以使用RNN创建字符级语言模型。
* 我们可以用困惑来评估语言模型的质量。

## 练习

1. 如果我们使用RNN预测文本序列中的下一个字符，任何输出所需的维度是多少？
2. 为什么RNN可以基于文本序列中的所有先前令牌在某个时间步表示令牌的条件概率？
3. 如果通过长序列反向传播，梯度会发生什么变化？
4. 与本节描述的语言模型相关的一些问题是什么？
